In [1]:
import tqdm

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', 100)
import glob

import os
from datetime import date, timedelta

import glob
import pandas as pd

glued_data = pd.DataFrame()
for file_name in glob.glob(directoryPath+'*.csv'):
    x = pd.read_csv(file_name, low_memory=False)
    glued_data = pd.concat([glued_data,x],axis=0)

In [2]:
path = '/Users/LyubomirKushev/Desktop/Data Science/Project/Jobs/Employment-Data/data_jobs_20210909/'

glued_data = pd.DataFrame()
for file_name in glob.glob(path+'*.csv'):
    x = pd.read_csv(file_name)
    glued_data = pd.concat([glued_data,x],axis=0)
    
glued_data.reset_index(inplace=True)
glued_data

,index,position_id,position_url,company_name,position_views,job_description,date_posted,position_rating,rating_color,position,benefits,technologies
0,0,5876034,https://www.jobs.bg/job/5876034,DataArt Europe Ltd,6,html About the vacancy Our client is a London-based startup that provides software for onlin...,27.08.21,NaN,NaN,"Senior React.js Engineer, Online Events Platform",София; Възможност за работа от вкъщи,"Ruby, JavaScript, React, jQuery, Ruby on Rails, HTML/CSS"
1,1,5877494,https://www.jobs.bg/job/5877494,ГРАФИКСОФТ ООД,15,html Software Engineer Grafixoft has 23 years of experience in software engineering and we a...,27.08.21,NaN,NaN,Software Engineer,София,"C#, Java, JavaScript, C/C++"
2,2,5837867,https://www.jobs.bg/job/5837867,SAP Labs Bulgaria Ltd.,10,html Requisition ID: 278246 Work Area: Software-Development Operations Expected Travel: 0 - ...,27.08.21,NaN,NaN,Database Cloud Engineer,София,SAP
3,3,5877489,https://www.jobs.bg/job/5877489,ГРАФИКСОФТ ООД,15,html Grafixoft has 23 years of experience in software engineering and we are most proud of t...,27.08.21,NaN,NaN,System Administrator – ProdOps Engineer,София,"AWS, Windows, Linux"
4,4,5837874,https://www.jobs.bg/job/5837874,SAP Labs Bulgaria Ltd.,7,html What we offer Our company culture is focused on helping our employees enable innovation...,27.08.21,NaN,NaN,Architect for Productive S4/HANA Automated Deployment into Public Cloud,София,"Java, Python, JavaScript, Spring, HTML/CSS, SQL, SAP, Azure, AWS, GCP, Ansible"
...,...,...,...,...,...,...,...,...,...,...,...,...
5848,1480,5876035,https://www.jobs.bg/job/5876035,DataArt Europe Ltd,7,html About the vacancy Our client is an EU-based crypto custodian company that provides solu...,27.08.21,NaN,NaN,"Senior Python Developer, Cryptocurrency",София,"Python, PostgreSQL, SQL, Docker, AWS, Kubernetes"
5849,1481,5837856,https://www.jobs.bg/job/5837856,SAP Labs Bulgaria Ltd.,8,html What we offer Our company culture is focused on helping our employees enable innovation...,27.08.21,NaN,NaN,Developer for BTP Core Platform Security Engineering,София,"Golang, Node.js, SAP, Kubernetes"
5850,1482,5876071,https://www.jobs.bg/job/5876071,ДжемСийк Кънсълтинг ЕООД,5,html We are advisors for growth. GemSeek Consulting Ltd. is international fast-growing marke...,27.08.21,NaN,NaN,MedTech/Senior Sales Consultant,София,Excel
5851,1483,5876120,https://www.jobs.bg/job/5876120,Appolica Ltd.,7,html Appolica is a leading software development company with a focus on mobile applications....,27.08.21,NaN,NaN,iOS Developer,София; Възможност за работа от вкъщи,"iOS, C/C++, Objective-C, Swift"


In [3]:
print('Input data has {} rows nad {} columns'
     .format(len(glued_data),
            len(glued_data.columns)))

Input data has 5853 rows nad 12 columns


##### Explor the dataset: nulls

In [4]:
for i,v in glued_data.iteritems():
    print('Number of nulls per {}: {}'.format(i, glued_data[i].isnull().sum()))

Number of nulls per index: 0
Number of nulls per position_id: 0
Number of nulls per position_url: 0
Number of nulls per company_name: 0
Number of nulls per position_views: 0
Number of nulls per job_description: 0
Number of nulls per date_posted: 0
Number of nulls per position_rating: 2749
Number of nulls per rating_color: 2749
Number of nulls per position: 0
Number of nulls per benefits: 0
Number of nulls per technologies: 850


In [7]:
glued_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5853 entries, 0 to 5852
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   index            5853 non-null   int64  
 1   position_id      5853 non-null   int64  
 2   position_url     5853 non-null   object 
 3   company_name     5853 non-null   object 
 4   position_views   5853 non-null   int64  
 5   job_description  5853 non-null   object 
 6   date_posted      5853 non-null   object 
 7   position_rating  3104 non-null   float64
 8   rating_color     3104 non-null   object 
 9   position         5853 non-null   object 
 10  benefits         5853 non-null   object 
 11  technologies     5003 non-null   object 
dtypes: float64(1), int64(3), object(8)
memory usage: 548.8+ KB


In [48]:
descriptions = glued_data[['position_id', 'job_description']]
descriptions.sample(5)

,position_id,job_description
3066,5859671,html ABOUT US SOFTWARE DEVELOPMENT & INNOVATIONS COMPANY We are a B2B company tha...
5353,5852613,Специалист системи за проследяване в автомобилният спорт \n\n\n\n\n\t...
2298,5858842,html Senior DevOps / Site Reliability Engineer We are looking for a Senior DevOps...
1386,5805771,html VMware is a global leader in cloud infrastructure and business mobility. VMware acceler...
3850,5888894,html Remote/On-site Senior Software Developer React/ Angular/ Vue/ Node Note: We of...


##### Clean Text: Remove punctuation, Tokenize text, and Remove Stopwords

In [46]:
import string
import re 
import nltk
stopword = nltk.corpus.stopwords.words('english')

def clean_text(text):
    text = text.lstrip('html ') # use to remove 'html' from begging of string; fixed in the scraper code
    text = "".join([char for char in text if char not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [word for word in tokens if word not in stopword]
    return text

descriptions['cleaned_text'] = descriptions['job_description'].apply(lambda x: clean_text(x.lower()))
descriptions.head()

<ipython-input-46-4d3215f25cf6>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  descriptions['cleaned_text'] = descriptions['job_description'].apply(lambda x: clean_text(x.lower()))


,position_id,job_description,cleaned_text
0,5876034,html About the vacancy Our client is a London-based startup that provides software for onlin...,"[vacancy, client, londonbased, startup, provides, software, online, events, virtual, events, tec..."
1,5877494,html Software Engineer Grafixoft has 23 years of experience in software engineering and we a...,"[software, engineer, grafixoft, 23, years, experience, software, engineering, proud, longlasting..."
2,5837867,html Requisition ID: 278246 Work Area: Software-Development Operations Expected Travel: 0 - ...,"[requisition, id, 278246, work, area, softwaredevelopment, operations, expected, travel, 0, 10, ..."
3,5877489,html Grafixoft has 23 years of experience in software engineering and we are most proud of t...,"[grafixoft, 23, years, experience, software, engineering, proud, longlasting, partnerships, esta..."
4,5837874,html What we offer Our company culture is focused on helping our employees enable innovation...,"[offer, company, culture, focused, helping, employees, enable, innovation, building, breakthroug..."
